<a href="https://colab.research.google.com/github/ShadrackOdikara/CALL-PHENO-TO-SNP-DATA/blob/main/CombiningT5andMPNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install huggingface_hub
!pip install datasets
!pip install -q accelerate==0.21.0 peft==0.13.2 bitsandbytes==0.40.2 transformers==4.46.2 trl==0.12.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
ERROR: Cannot install accelerate==0.21.0, peft==0.13.2 and trl==0.12.0 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible

In [5]:
!pip install --upgrade peft
!pip install --upgrade trl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 18.5 MB/s eta 0:00:00


In [8]:
"""This notebook is design to train and evaluate an AI based on T5 architecture and fine tuned
for chat and Text generation and for sequnce classifcation. We train Llama3 for causalModel generation
while the BERT roBERTa will be fine tuned sor scoring or rather sequence classification"""

import os
import torch
import numpy as np
import pandas as pd
from trl import SFTTrainer
#import sentencepiece as spm
from datasets import load_dataset
from transformers import AutoTokenizer
from peft import LoraConfig, get_peft_model
from torch.utils.data import Dataset, DataLoader
from transformers import (AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, Trainer, TrainingArguments, pipeline, logging)

In [9]:
from google.colab import userdata
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")

In [10]:
# we store our path to where our dataset is stored on kaggle in ReportSDGDatapath
# From this path we will be able to load our dataset to the environment since they
# are linked.
# we store our path to where our dataset is stored on kaggle in ReportSDGDatapath
# From this path we will be able to load our dataset to the environment since they
# are linked.

ReportSDGDatapath = '/content/drive/MyDrive/FutureProofingAgri/curated_agric_dataset_v2.csv'
InputResponseDatapath = '/content/drive/MyDrive/FutureProofingAgri/QA_data.csv'
PotatoDiseaseDatapath = '/content/drive/MyDrive/FutureProofingAgri/potatoDiseaseprofile.csv'

In [11]:
# We load our dataset which is in the path stored in ReportSDGDatapath using
# pandas  read_csv. This is stored as a dataframe in object ReportSDGData
ReportSDGData = pd.read_csv(ReportSDGDatapath)
InputResponseData = pd.read_csv(InputResponseDatapath)
PotatoDiseaseData = pd.read_csv(PotatoDiseaseDatapath)

# We adjust and edit our dataset in the Dataframe object by specifying the labels
# and converting the labled into string values so that it can be processed smulteneuosly with
# the other columns of the datase. We have three columns so far 'premise', 'hypothesis', and 'label'
ReportSDGData['label'] = ReportSDGData['label'].replace({'0': 'neutral', '1': 'entailment', '2': 'contradiction'})
InputResponseData.rename(columns = {'response': 'premise', 'input': 'hypothesis'}, inplace=True)

InputResponseData = InputResponseData[['premise', 'hypothesis']]

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset
import torch
#import torch_xla
#import torch_xla.core.xla_model as xm
#import sentencepiece

# Ensure all entries are strings
ReportSDGData['premise'] = ReportSDGData['premise'].apply(lambda x: ' '.join(x) if isinstance(x, list) else str(x))
InputResponseData['premise'] = InputResponseData['premise'].apply(lambda x: ' '.join(x) if isinstance(x, list) else str(x))

ReportSDGDataCombined = pd.concat([ReportSDGData, InputResponseData], axis = 0)

# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(ReportSDGDataCombined)

# Prepare inputs and targets for T5
dataset = dataset.map(lambda example: {
    'input_text': 'storytelling prompt: ' + example['premise'],
    'target_text': example['premise']
})

# Extract inputs and targets
inputs = dataset['input_text']
targets = dataset['target_text']

# Load the model and tokenizer
gen_model_name = "t5-base"
gen_tokenizer = T5Tokenizer.from_pretrained(gen_model_name)
gen_model = T5ForConditionalGeneration.from_pretrained(gen_model_name)


# Fix vocab_size mismatch
#gen_model.resize_token_embeddings(len(gen_tokenizer))


########################################################################################
################################LoRA configuration######################################
########################################################################################
# Apply LoRA configuration
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    task_type="SEQ2SEQ_LM",  # Sequence-to-sequence task for T5
    r=16,  # Rank of the low-rank matrices
    lora_alpha=32,  # Scaling factor
    lora_dropout=0.1  # Dropout for LoRA
)

# Wrap the model with LoRA
gen_model = get_peft_model(gen_model, lora_config)

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
gen_model.to(device)

# Set device to TPU
#device = xm.xla_device()

# Move model to TPU
#gen_model.to(device)

print("LoRA model ready for fine-tuning.")


#########################################################################################

# Tokenize the inputs and targets
inputs = gen_tokenizer(inputs, padding="max_length", truncation=True, return_tensors="pt", max_length=512)
targets = gen_tokenizer(targets, padding="max_length", truncation=True, return_tensors="pt", max_length=512)

# Move model to GPU if available
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#gen_model.to(device)

print("Model and tokenizer ready for fine-tuning.")


Map:   0%|          | 0/56497 [00:00<?, ? examples/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

LoRA model ready for fine-tuning.


In [14]:
from torch.utils.data import Dataset, DataLoader

class StoryDataset(Dataset):
    def __init__(self, inputs, targets):
        self.inputs = inputs
        self.targets = targets

    def __len__(self):
        return len(self.inputs["input_ids"])

    def __getitem__(self, idx):
        input_ids = self.inputs["input_ids"][idx]
        attention_mask = self.inputs["attention_mask"][idx]
        labels = self.targets["input_ids"][idx]

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels,
        }

# Initialize dataset and dataloader
train_dataset = StoryDataset(inputs, targets)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

In [ ]:
from transformers import AdamW
import torch

# Define the optimizer
optimizer = AdamW(gen_model.parameters(), lr=2e-5)

# Training loop
epochs = 2
for epoch in range(epochs):
    gen_model.train()
    for batch in train_loader:
        input_ids = batch["input_ids"].to('cuda')
        attention_mask = batch["attention_mask"].to('cuda')
        labels = batch["labels"].to('cuda')

        # Forward pass
        outputs = gen_model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Epoch: {epoch}, Loss: {loss.item()}")

# Save the fine-tuned model and tokenizer
gen_model.save_pretrained("./t5_potatoAgric_model")
gen_tokenizer.save_pretrained("./t5_potatoAgric_model")

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch: 0, Loss: 14.89366626739502
Epoch: 0, Loss: 14.718505859375
Epoch: 0, Loss: 16.910083770751953
Epoch: 0, Loss: 19.963483810424805
Epoch: 0, Loss: 17.870588302612305
Epoch: 0, Loss: 17.88780975341797
Epoch: 0, Loss: 15.969255447387695
Epoch: 0, Loss: 20.167545318603516
Epoch: 0, Loss: 19.728715896606445
Epoch: 0, Loss: 16.064956665039062
Epoch: 0, Loss: 15.492681503295898
Epoch: 0, Loss: 21.750980377197266
Epoch: 0, Loss: 16.464946746826172
Epoch: 0, Loss: 16.20940589904785
Epoch: 0, Loss: 17.127408981323242
Epoch: 0, Loss: 15.415314674377441
Epoch: 0, Loss: 16.67934226989746
Epoch: 0, Loss: 15.452330589294434
Epoch: 0, Loss: 17.08479118347168
Epoch: 0, Loss: 19.84595489501953
Epoch: 0, Loss: 18.336610794067383
Epoch: 0, Loss: 12.954341888427734
Epoch: 0, Loss: 19.93832778930664
Epoch: 0, Loss: 16.19782066345215
Epoch: 0, Loss: 19.322052001953125
Epoch: 0, Loss: 15.255996704101562
Epoch: 0, Loss: 15.921335220336914
Epoch: 0, Loss: 13.752740859985352
Epoch: 0, Loss: 16.213291168212